In [1]:
from pymongo import MongoClient
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import *
from konlpy.utils import pprint
import numpy as np
import lda
from sklearn.pipeline import Pipeline
import pandas as pd
# import statsmodels.api as sm
from collections import Counter

In [3]:
twitter = Twitter()

/home/ubuntu/.local/share/virtualenvs/zaldata-rMQHoK9R/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
english = re.compile(r'[a-zA-Z0-9]')

In [5]:
korean = re.compile(r'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]')

In [3]:
mongo = MongoClient('13.125.29.252', 27017)
review = mongo.zaldata.restaurant

# 서버에서 DB 불러오기

In [2]:
mongo = MongoClient('13.125.29.252', 27017)
review = mongo.zaldata.restaurant

In [3]:
pipelines = []
pipelines.append({'$group': {'_id': '$name','review':{'$addToSet':'$review'}}})
result = review.aggregate(pipelines)

In [4]:
review_list = []
for doc in result:
    review_list.append(doc)

In [5]:
print(len(review_list))

1269


In [7]:
review_list[0]

{'_id': '흑돈가 제주본점',
 'review': ['크고 유명한 레스토랑 이지만 아래 방법보다 훨씬 (최소한 관광객 중) 기대 이상이었습니다. ',
  '가게가 유명한 곳이 있다. 우린 3 다양한 종류의 고기, 제주 검은 돼지고기, 절인 돼지 고기 및 돼지 고기 여밈부분의. 관대 한 부분은, 직원들은 친절하고 효율적. 2명을 위한 정보 S $60 달러. ',
  '이 곳의 추천에 따라 우리의 투어 가이드, 음식은 좋았습니다. 이것은 우리의 가이드였던 경우 를 알고 영어로 의사소통. 솔직히 말하자면, 서비스는 전형적인 한국식 을 요리 할 때 서비스 기능으로, 고기를 잘라 먹고 최대 수 있는지 결정합니다. 구운 고기와 같은 경우, 식용 으로 보이지 않는 말을 할 수 있는 되었습니다. 이 때문에 어떤 사람들은 서비스를 할 수 있습니다. 이에 신경 쓰지 않는다면, 정말 좋은 사람이 아니라면 - 그래서 - 무례한 서비스 하는 곳이 아닙니다.',
  '여기에 대한 좋은 리뷰를 많이 읽어서 한 번 먹어보러 갔다. 2층으로 되어 있고 크다. 우린 저녁 먹으로 갔는데 그날 사람 많았다. 숯을 쓰는데 진짜 향기좋고 식감 좋다. 너무 오래 구우면 고기가 타 버리니 주의해라. 양배추, 양파, 김치로 싸 먹어라. 완전 행복해진다 하하. 지금 이거 쓰면서도 벌써 또 생각난다!',
  '서울보다 맛있고 크기도 크고가격도 조금 저렴합니다. 흑돈가의 멸치젓이 제일이네요밑반찬도 푸짐합니다. 게장도 나오고',
  '제주도 갈떄 마다 방문하는 곳입니다. 지난번 방문때는 이틀연속으로 갔었습니다.액젓에 찍어먹는것도 맛있도 도톰한 고기에 나오는 육즙도 맛이 좋았습니다.크게 친절하지는 않지만 식사는 만족하는 곳입니다.',
  '사람많고 많이 기다려야하지만그래도 맛있습니다..제주도 가면 꼭 들리는 맛집이지요.서비스는 아주 만족스럽지 않지만 그래도 맛은있으니..',
  '소문듣고 찾아서갔는데 소문만큼 미친듯이 맛있는건아니네요. 돼지고기를 스테이크처럼 약간 덜익혀서주는데 처음먹는 특이한 맛이었습니다.


# 전처리 - stemming

In [11]:
def do_stemming(reviews):
    result_list = []
    for i in range(len(reviews)):
        a = re.sub(' ','s',reviews[i])    #띄어쓰기 위해 s 붙힘
        b = twitter.pos(a, stem = True, norm= True)  #형태소 분석
        string = ''
        for word in b:                #문장으로 합침
            string += word[0]
        c = re.sub('s',' ',string)    #띄어쓰기 분리
        result_list.append(c)
        print(result_list)
    return result_list

In [12]:
# 불용어 불러오기

f = open('stopwords.txt')
stopwords = set([l.strip() for l in f.readlines()])

f.close()

stopwords

{'가',
 '가까스로',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '갖고말하자면',
 '같다',
 '같이',
 '개의치않고',
 '거니와',
 '거바',
 '거의',
 '것',
 '것과 같이',
 '것들',
 '게다가',
 '게우다',
 '겨우',
 '견지에서',
 '결과에 이르다',
 '결국',
 '결론을 낼 수 있다',
 '겸사겸사',
 '고려하면',
 '고로',
 '곧',
 '공동으로',
 '과',
 '과연',
 '관계가 있다',
 '관계없이',
 '관련이 있다',
 '관하여',
 '관한',
 '관해서는',
 '구',
 '구체적으로',
 '구토하다',
 '그',
 '그들',
 '그때',
 '그래',
 '그래도',
 '그래서',
 '그러나',
 '그러니',
 '그러니까',
 '그러면',
 '그러므로',
 '그러한즉',
 '그런 까닭에',
 '그런데',
 '그런즉',
 '그럼',
 '그럼에도 불구하고',
 '그렇게 함으로써',
 '그렇지',
 '그렇지 않다면',
 '그렇지 않으면',
 '그렇지만',
 '그렇지않으면',
 '그리고',
 '그리하여',
 '그만이다',
 '그에 따르는',
 '그위에',
 '그저',
 '그중에서',
 '그치지 않다',
 '근거로',
 '근거하여',
 '기대여',
 '기점으로',
 '기준으로',
 '기타',
 '까닭으로',
 '까악',
 '까지',
 '까지 미치다',
 '까지도',
 '꽈당',
 '끙끙',
 '끼익',
 '나',
 '나머지는',
 '남들',
 '남짓',
 '너',
 '너희',
 '너희들',
 '네',
 '넷',
 '년',
 '논하지 않다',
 '놀라다',
 '누가 알겠는가',
 '누구',
 '다른',
 '다른 방면으로',
 '다만',
 '다섯',
 '다소',
 '다수',
 '다시 말하자면',
 '다시말하면',
 '다음',
 '다음에',
 '다음으로',
 '단지',
 '답다',
 '당신',
 '당장',
 '대로 하다',
 '대하면',
 '대하여',
 '대해 말하자면',
 '대해서',
 '댕그',


리뷰가 12만개에 달하는 관계로 멀티프로세스로 처리함

In [13]:
print(twitter.morphs('이것도 되나욬ㅋㅋㅋㅋㅋ', norm=True, stem=True))
print(twitter.pos('이것도 되나욬ㅋㅋㅋㅋ', norm=True))

['이', '것', '도', '되다', 'ㅋㅋㅋ']
[('이', 'Determiner'), ('것', 'Noun'), ('도', 'Josa'), ('되나요', 'Verb'), ('ㅋㅋㅋ', 'KoreanParticle')]


In [14]:
def perform(review):
    text = re.sub('[^가-힣]', ' ', review)
    tokens = twitter.morphs(text, norm=True, stem=True)
    tokens = [token for token in tokens if token not in stopwords]
    text = ' '.join(tokens)
    return text
    

In [15]:
from multiprocessing import Pool
from tqdm import tqdm_notebook


def perform(review):
    text = re.sub('[^가-힣]', ' ', review)
    tokens = twitter.morphs(text, norm=True, stem=True)
    tokens = [token for token in tokens if token not in stopwords]
    text = ' '.join(tokens)
    return text
    


def preprocess(reviews):
    for review_dic in tqdm_notebook(reviews):
        review_list = review_dic['review']
        new_review_list = []
        for review in review_list:
            new_review_list.append(perform(review))
            
        review_dic['review'] = new_review_list
    
preprocess(review_list)

KeyboardInterrupt: 

In [4]:
preprocessed = mongo.zaldata.text

In [5]:
pipelines = []
pipelines.append({'$group': {'_id': '$name','review':{'$addToSet':'$review'}}})
result = preprocessed.aggregate(pipelines)

In [6]:
review_list = []
for doc in result:
    review_list.append(doc)

In [7]:
len(review_list)

1269

In [11]:
review_list[0]['review'][0]

'째 꼬바 꼬박 찾다 찻집 이다 대추차 매우 매우 추천 드림 메뉴 도 상당하다 많이 시도 하다 찻집 이에요 전통 나름 개선 많이 해 가다 자리다 대부분 바닥 이라 편하다 않다'

In [23]:
service_good_feature = {'서비스':['좋','친절','괜찮','최고','빠르','짱','훌륭','추천','감사','구수','최상','대박','훈훈','특별','개이득','최고','만족','세련','최고','감동'],'사장':['친절','스윗','센스'],'알바':['친절','스윗','센스'],'직원': ['친절','스윗','센스']}

In [24]:
service_bad_feature = {'서비스': ['아쉽','최악','나쁘','느리','빡치','비추','별로','그냥','낙제','쏘다쏘다','엉망','실망','불친절','문제','컴플레인','거지','그닥','그다지','구려','불편','엉성','헬','개판'],'알바':['불친절','똑바로','재수'],'사장':['불친절','똑바로','재수'],'직원':['불친절','똑바로','재수']}

In [25]:
taste_good_feature = {'간이':['맞','적절','딱','환상','담백'],'간도':['맞','적절','딱','환상','담백'],'음식':['깔끔'],'맛':['있']}

In [26]:
taste_bad_feature = {'간이':['세','쎄','강하다','별로'],'간도':['세','쎄','강하다','별로'],'음식':['별로','쏘다쏘다','최악'],'맛':['별로','최악']}

In [27]:
taste_good_emotion = ['고소','부드럽','신선','촉촉','싱싱','정갈']

In [28]:
taste_bad_emotion = ['싱겁','느끼다하다','짜다','느끼다','짜다','딱딱하다','차갑다']

In [29]:
cost_good_feature = {'가격': ['괜찮','착하다','저렴','적당','싸다','좋다','합리적','훌륭','최고','만족','마음','든든','알맞다','무난','괜춘','최상','최상','굿','엄지','낮'],'가성비':['괜찮','착하다','저렴','적당','싸다','좋다','합리적','훌륭','최고','만족','마음','든든','알맞다','무난','괜춘','최상','최상','굿','엄지'],'양도':['많','적당','푸다짐하고','괜찮다','넉넉','충분','든든'],'양이':['많','적당','푸다짐하고','괜찮다','넉넉','충분','든든'],'양은':['많','적당','푸다짐하고','괜찮다','넉넉','충분','든든']}

In [30]:
cost_bad_feature ={'가격': ['비싸','있다','나쁘','사악','비효율','높다','부담','아쉽','쏘다쏘다','별로','그닥','그다지','쎄','ㅎㄷㄷ','높','거품'],'가성비':['별로'],'양이':['적다','작다','아쉽','적다','다소','별로'],'양은':['적다','작다','아쉽','적다','다소','별로'],'양도':['적다','작다','아쉽','적다','다소','별로']}

In [31]:
atmosphere_good_feature = {'분위기': ['좋','괜찮','조용','깔끔','적당','깡패','굉장','아담','완벽','아기자기','고급','최고','세련','만족','아늑','훌륭','예쁘','이쁘','짱','심쿵','따뜻','깨끗','독특','매력','모던','취향저격','맘','마음','클래식','아름','인상적','귀엽','포근'], '인테리어': ['좋','괜찮','조용','깔끔','적당','깡패','굉장','아담','완벽','아기자기','고급','최고','세련','만족','아늑','훌륭','예쁘','이쁘','짱','심쿵','따뜻','깨끗','독특','매력','모던','취향저격','맘','마음','클래식','아름','인상적','귀엽','포근']}

In [32]:
atmosphere_bad_feature = {'분위기': ['나쁘다','바쁘다','어수선하다','이상하다','촌스럽다','별로','부담스럽다','시끄럽','복잡' ],'인테리어':[]}

In [33]:
visit_good_feature = {'의사':['있','충만','백프로','백프롭','많','만땅','마구','그득','만점','넘침'],'다시':['가다','오다','방문','찾다','가보다','한번','갈다','찾아가다','가야지','갈거다','방문하다보고','생각나다','방문한다면','와보고','재방문','접하다','간다면','갈다때가','먹다고프다','방문한다임','오자고','가기로','갈다생각이다','가면'],'굳이':[]}

In [34]:
visit_bad_feature = {'의사':['글쎄'],'굳이':['다시','많이','여기까지','줄서서','찾아','시키다','가다','찾다','여기','기다리다','줄을','사먹'],'다시':[]}

In [60]:
features = mongo.zaldata.features
features.drop()

In [61]:
features.insert_one({'name': 'service_good', 'features': service_good_feature})
features.insert_one({'name':'service_bad', 'features': service_bad_feature})
features.insert_one({'name':'taste_good', 'features': taste_good_feature})
features.insert_one({'name':'taste_bad', 'features': taste_bad_feature})
features.insert_one({'name':'cost_good', 'features': cost_good_feature})
features.insert_one({'name':'cost_bad', 'features': cost_bad_feature})
features.insert_one({'name':'atmosphere_good', 'features': atmosphere_good_feature})
features.insert_one({'name':'atmosphere_bad', 'features': atmosphere_bad_feature})
features.insert_one({'name':'visit_good', 'features': visit_good_feature})
features.insert_one({'name':'visit_bad', 'features': visit_bad_feature})

In [62]:
f = mongo.zaldata.features.find()
for i in f:
    print(i)

{'_id': ObjectId('5cf76bae86aa7106693fc076'), 'name': 'service_good', 'features': {'서비스': ['좋', '친절', '괜찮', '최고', '빠르', '짱', '훌륭', '추천', '감사', '구수', '최상', '대박', '훈훈', '특별', '개이득', '최고', '만족', '세련', '최고', '감동'], '사장': ['친절', '스윗', '센스'], '알바': ['친절', '스윗', '센스'], '직원': ['친절', '스윗', '센스'], '_id': ObjectId('5cf7489086aa7106693fc061')}}
{'_id': ObjectId('5cf76bae86aa7106693fc077'), 'name': 'service_bad', 'features': {'서비스': ['아쉽', '최악', '나쁘', '느리', '빡치', '비추', '별로', '그냥', '낙제', '쏘다쏘다', '엉망', '실망', '불친절', '문제', '컴플레인', '거지', '그닥', '그다지', '구려', '불편', '엉성', '헬', '개판'], '알바': ['불친절', '똑바로', '재수'], '사장': ['불친절', '똑바로', '재수'], '직원': ['불친절', '똑바로', '재수']}}
{'_id': ObjectId('5cf76bae86aa7106693fc078'), 'name': 'taste_good', 'features': {'간이': ['맞', '적절', '딱', '환상', '담백'], '간도': ['맞', '적절', '딱', '환상', '담백'], '음식': ['깔끔'], '맛': ['있']}}
{'_id': ObjectId('5cf76bae86aa7106693fc079'), 'name': 'taste_bad', 'features': {'간이': ['세', '쎄', '강하다', '별로'], '간도': ['세', '쎄', '강하다', '별로'], '음식': ['별로', '쏘다쏘다', '최악

In [59]:
features.find_one({'name': 'service_good'})['features'].pop('_id')

ObjectId('5cf7489086aa7106693fc061')

In [24]:
negative_word_emotion = ['안','않','못','없','아닌','아니']

In [25]:
def get_feature_keywords(feature_keywords, review):
    feature_temp = []
    for keyword in feature_keywords:
        if re.findall(keyword, review):
            a = re.findall(keyword +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+', review)
            b = re.findall(keyword + '+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review)
            c = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + keyword +'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review)
            
            for ngram in a:
                t = ()
                feature_temp.append(t + (ngram,keyword))
            for ngram in b:
                t = ()
                feature_temp.append(t + (ngram,keyword))
            for ngram in c:
                t = ()
                feature_temp.append(t + (ngram,keyword))

    return feature_temp

In [26]:
def get_feature_emotions(feature_good_dict,feature_bad_dict, feature_temp):
    good_feature_emotion_list = []
    bad_feature_emotion_list = []
    
    for ngrams in feature_temp:
        keyword = ngrams[1]
        ngram = ngrams[0]
        is_bad_feature = None
        
        good_emotion_list = feature_good_dict[keyword]
        bad_emotion_list = feature_bad_dict[keyword]
        for emotion in good_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = False  
        for emotion in bad_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = True    
        for negative in negative_word_emotion:
            if re.findall(negative, ngram):
                if is_bad_feature == True:
                    is_bad_feature = False
                    break
                elif is_bad_feature == False:
                    is_bad_feature = True
                    break
                else:
                    is_bad_feature = True
                    break   
        if is_bad_feature:
            bad_feature_emotion_list.append(ngram)
        elif is_bad_feature == False:
            good_feature_emotion_list.append(ngram)
        else:
            pass
    return good_feature_emotion_list, bad_feature_emotion_list

In [27]:
def get_taste_emotion(taste_good_emotions, taste_bad_emotions):
    bad_taste_emotion_list = []
    good_taste_emotion_list = []
    for ngrams in taste_good_emotions:
        ngram = ngrams[0]
        is_bad_taste = False
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_taste = True
        if is_bad_taste:
            bad_taste_emotion_list.append(ngram)
        else:
            good_taste_emotion_list.append(ngram)
    
    for ngrams in taste_bad_emotions:
        ngram = ngrams[0]
        is_bad_taste = True
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_taste = False
        if is_bad_taste:
            bad_taste_emotion_list.append(ngram)
        else:
            good_taste_emotion_list.append(ngram)
            
    return good_taste_emotion_list, bad_taste_emotion_list

# Test 3. 음식점 별로 속성별 감정지수 파악

In [28]:
check_division = lambda x, y: y if y ==0 else round((x / float(y)),2)

result = []

for i, reviews in enumerate(review_list):
    restaurant_good_service_count = 0
    restaurant_bad_service_count = 0
    restaurant_good_atmosphere_count =0
    restaurant_bad_atmosphere_count =0
    restaurant_good_cost_count =0
    restaurant_bad_cost_count =0
    restaurant_good_visit_count = 0
    restaurant_bad_visit_count = 0
    restaurant_good_taste_count = 0
    restaurant_bad_taste_count = 0
    print (i, reviews['_id'])
    
    for j, review in enumerate(reviews['review']):
        #print review
        service_temp = get_feature_keywords(service_good_feature.keys(), review)
        good_service,bad_service = get_feature_emotions(service_good_feature, service_bad_feature, service_temp)

        atmosphere_temp = get_feature_keywords(atmosphere_good_feature.keys(), review)
        good_atmosphere,bad_atmosphere = get_feature_emotions(atmosphere_good_feature, atmosphere_bad_feature, atmosphere_temp)

        cost_temp = get_feature_keywords(cost_good_feature.keys(), review)
        good_cost,bad_cost = get_feature_emotions(cost_good_feature, cost_bad_feature, cost_temp)

        visit_temp = get_feature_keywords(visit_good_feature.keys(), review)
        good_visit,bad_visit = get_feature_emotions(visit_good_feature, visit_bad_feature, visit_temp)

        taste_temp = get_feature_keywords(taste_good_feature.keys(), review)
        good_taste,bad_taste = get_feature_emotions(taste_good_feature, taste_bad_feature, taste_temp)
        taste_good_emotion_temp = get_feature_keywords(taste_good_emotion, review)
        taste_bad_emotion_temp = get_feature_keywords(taste_bad_emotion, review)
        good_taste2, bad_taste2 = get_taste_emotion(taste_good_emotion_temp,taste_bad_emotion_temp)
        good_taste.extend(good_taste2)
        bad_taste.extend(bad_taste2)

        if len(good_service) > len(bad_service):
            restaurant_good_service_count += 1
        elif len(good_service) < len(bad_service):
            restaurant_bad_service_count += 1
        else:
            pass
        
        if len(good_atmosphere) > len(bad_atmosphere):
            restaurant_good_atmosphere_count += 1
        elif len(good_atmosphere) < len(bad_atmosphere):
            restaurant_bad_atmosphere_count += 1
        else:
            pass
        
        if len(good_cost) > len(bad_cost):
            restaurant_good_cost_count += 1
        elif len(good_cost) < len(bad_cost):
            restaurant_bad_cost_count += 1
        else:
            pass
            
        if len(good_visit) > len(bad_visit):
            restaurant_good_visit_count += 1
        elif len(good_visit) < len(bad_visit):
            restaurant_bad_visit_count += 1
        else:
            pass
        
        if len(good_taste) > len(bad_taste):
            restaurant_good_taste_count += 1
        elif len(good_taste) < len(bad_taste):
            restaurant_bad_taste_count += 1
        else:
            pass
    
    
    result.append({'name': reviews['_id'],
                   'restaurant_good_visit_count': restaurant_good_visit_count,
                   'restaurant_bad_visit_count': restaurant_bad_visit_count,
                   'restaurant_good_service_count': restaurant_good_service_count,
                   'restaurant_bad_service_count': restaurant_bad_service_count,
                   'restaurant_good_atmosphere_count': restaurant_good_atmosphere_count,
                   'restaurant_bad_atmosphere_count': restaurant_bad_atmosphere_count,
                   'restaurant_good_cost_count': restaurant_good_cost_count,
                   'restaurant_bad_cost_count': restaurant_bad_cost_count,
                   'restaurant_good_taste_count': restaurant_good_taste_count,
                   'restaurant_bad_taste_count': restaurant_bad_taste_count
                  })
#     print('Total review count: {}'.format(len(review_list)))
#     print('Good service: {}/{} = {}'.format(restaurant_good_service_count,restaurant_good_service_count + restaurant_bad_service_count,check_division(restaurant_good_service_count, restaurant_good_service_count + restaurant_bad_service_count) ))
#     print('Bad service: {}/{} = {}'.format(restaurant_bad_service_count,restaurant_good_service_count + restaurant_bad_service_count, check_division(restaurant_bad_service_count, restaurant_good_service_count + restaurant_bad_service_count)))
#     print('Good atmosphere: {}/{} = {}'.format(restaurant_bad_service_count,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count,check_division(restaurant_good_atmosphere_count,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count))) 
#     print('Bad atmosphere: {}/{} = {}'.format(restaurant_bad_atmosphere_count,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count, check_division(restaurant_bad_atmosphere_count ,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count)))
#     print('Good cost: {}/{} = {}'.format(restaurant_good_cost_count,restaurant_good_cost_count + restaurant_bad_cost_count, check_division(restaurant_good_cost_count,restaurant_good_cost_count + restaurant_bad_cost_count)))
#     print('Bad cost: {}/{} = {}'.format(restaurant_bad_cost_count,restaurant_good_cost_count + restaurant_bad_cost_count, check_division(restaurant_bad_cost_count,restaurant_good_cost_count + restaurant_bad_cost_count)))
#     print('Good taste: {}/{} = {}'.format(restaurant_good_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count,check_division(restaurant_good_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count)))
#     print('Bad taste: {}/{} = {}'.format(restaurant_bad_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count,check_division(restaurant_bad_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count)))
#     print()

0 수연산방
1 어글리베이커리
2 몽카페그레고리
3 하루
4 서울동
5 배터리파크
6 마츠
7 메종드루즈
8 방켓
9 레스쁘아뒤이브
10 오아시스
11 유타로
12 금수복국 본점
13 태극당
14 고집돌우럭
15 마녀주방
16 뱃고동
17 더 파크뷰
18 99's 서양밥집
19 멘야산다이메
20 소나
21 스놉
22 호야초밥참치전문점
23 이재모피자
24 순대실록
25 동남아
26 청키면가
27 폴리스브릭오븐피제리아
28 브라더후드키친
29 빚짜
30 빌즈 광화문
31 백부장집닭한마리
32 모파상
33 필동면옥
34 이관우파스타
35 바다파스타
36 신토불이떡볶이
37 17도씨
38 푸하하크림빵
39 대포찜닭
40 계열사
41 실론살롱
42 맛찬들왕소금구이
43 스시시로
44 엘빈
45 남포면옥
46 버터밀크
47 정돈
48 서촌계단집
49 소호정
50 방콕야시장
51 군산집
52 굴드커피 로스터스
53 코이라멘
54 옥동식
55 돕감자탕전문점
56 카페진정성
57 메이드바이아린
58 보영만두
59 봉산옥
60 더파크뷰
61 키와마루아지
62 홍명
63 투뿔등심
64 샤이바나
65 젤라띠젤라띠
66 하동관 
67 봉피양
68 키키찬팅
69 별미곱창
70 이치젠
71 이품
72 스와니예
73 주
74 차이나당
75 목포집
76 힐사이드테이블
77 마얘
78 줄라이
79 장수삼
80 한일옥
81 우유니아이스크림
82 지파씨
83 호수집
84 몽중헌
85 송쉐프
86 장꼬방
87 도렐
88 셰막
89 예티
90 쟈니덤플링
91 라이너스 바베큐 앤 치킨
92 외래향
93 나마스까르 삼산점
94 더블트러블
95 딩딤1968
96 리틀앤머치
97 하하
98 케르반레스토랑
99 장서는날
100 제주몬트락
101 오설록티하우스
102 더은교
103 오다가다
104 소년식당
105 들름집
106 지구당 
107 한잔의추억
108 교남김밥
109 코다차야
110 반서울
111 한남북엇국
112 마초 그릴
113 콘지하우스 명동
114 바비레드 강남
115 인디언 커리 하우스 빈

834 브릭 오븐 뉴욕 피자
835 로로11
836 애플하우스
837 스카이온파이브다이닝
838 흑돈가 제주본점
839 평안도만두집
840 토끼정
841 미피아체
842 코코이찌방야
843 우동카덴
844 타코칠리칠리
845 프로간장게장
846 유자유김치떡볶이
847 가보정
848 베테랑
849 달밭소곱창
850 신선설농탕 명동점
851 대림창고
852 송도옥
853 피터팬 1978
854 지미스
855 글램라운지
856 고센
857 혜화칼국수
858 유라꾸키친
859 블랑
860 라디오베이
861 벽제 갈비
862 혼가츠
863 트레비아
864 파올로 데 마리아
865 타르틴베이커리
866 라공방
867 하시엔다
868 자리
869 광화문집
870 울프하운드 아이리쉬 펍 & 레스토랑
871 맘맘테이블
872 아우어베이커리
873 몽상클레르
874 더브레드블루
875 무르무르드구스토
876 안즈
877 우진해장국
878 이치에
879 빌즈 잠실
880 카네마야제면소
881 강남면옥
882 케이트앤케이크
883 신촌황소곱창
884 고냉지
885 잇탤리
886 스윗밸런스
887 산촌
888 닌스시
889 마포 진짜 원조 최대포
890 울프강스테이크하우스코리아
891 당고집
892 컨버세이션
893 브릭오븐
894 만푸쿠
895 문화식당
896 카페에이블
897 대도식당
898 개금밀면
899 팬아시아
900 마이쥬스
901 마렘마트라토리아
902 현대옥 남부시장점
903 가나안덕 일산본점
904 엣모스피어
905 띵똥와플
906 모모 카페
907 도쿄빙수
908 도림
909 트라토리아챠오
910 제임스 치즈 등갈비 홍대 본점
911 믹스앤몰트
912 황산샤브샤브
913 켄비멘리키
914 이태리국시
915 페닌슐라
916 단풍나무 집 이태원점
917 연남서식당
918 스펙터
919 앙토낭카렘
920 홍대마늘떡볶이
921 팩피
922 커피리브레
923 에머이
924 일젤라또
925 무라
926 김판조닭강정
927 오뗄두스
928 초당할머니순두부
929 나인로드피제리아

In [29]:
result[0]

{'name': '수연산방',
 'restaurant_good_visit_count': 2,
 'restaurant_bad_visit_count': 2,
 'restaurant_good_service_count': 0,
 'restaurant_bad_service_count': 0,
 'restaurant_good_atmosphere_count': 11,
 'restaurant_bad_atmosphere_count': 0,
 'restaurant_good_cost_count': 0,
 'restaurant_bad_cost_count': 7,
 'restaurant_good_taste_count': 13,
 'restaurant_bad_taste_count': 8}

In [3]:
stat = mongo.zaldata.statistic

In [23]:
r = stat.insert_many(result)
print(len(r.inserted_ids))

NameError: name 'stat' is not defined

In [10]:
from krwordrank.word import KRWordRank

wordrank = KRWordRank(min_count=5, max_length=10, verbose=True)


In [13]:
from tqdm import tqdm_notebook

for review in tqdm_notebook(review_list):
    while True:
        try:
            keywords, rank, graph = wordrank.extract(review['review'], 0.85, 10)
        except:
            continue
        print('5')
        break
        
    top = next(iter(keywords.keys()))
    print(top)
    
    mongo.zaldata.statistic.update_one({'name': review['_id']}, {'$set': {'top': top}})
    
    
    

scan vocabs ... 
num vocabs = 276
done = 10
5
좋다
scan vocabs ... 
num vocabs = 534
done = 10
5
크림
scan vocabs ... 
num vocabs = 1002
done = 10 Early stopped.
5
하다
scan vocabs ... 
num vocabs = 214
done = 10
5
먹다
scan vocabs ... 
num vocabs = 326
done = 10
5
하다
scan vocabs ... 
num vocabs = 235
done = 10
5
곱창
scan vocabs ... 
num vocabs = 401
done = 10
5
하다
scan vocabs ... 
num vocabs = 317
done = 10
5
마카롱
scan vocabs ... 
num vocabs = 226
done = 10
5
먹다
scan vocabs ... 
num vocabs = 400
done = 10
5
하다
scan vocabs ... 
num vocabs = 537
done = 10
5
하다
scan vocabs ... 
num vocabs = 410
done = 10
5
먹다
scan vocabs ... 
num vocabs = 420
done = 10
5
하다
scan vocabs ... 
num vocabs = 520
done = 10
5
하다
scan vocabs ... 
num vocabs = 527
done = 10
5
하다
scan vocabs ... 
num vocabs = 320
done = 10
5
하다
scan vocabs ... 
num vocabs = 177
done = 10
5
오징어
scan vocabs ... 
num vocabs = 916
done = 10 Early stopped.
5
하다
scan vocabs ... 
num vocabs = 207
done = 10
5
먹다
scan vocabs ... 
num vocabs = 933
do

done = 10
5
만두
scan vocabs ... 
num vocabs = 705
done = 10
5
하다
scan vocabs ... 
num vocabs = 461
done = 10
5
먹다
scan vocabs ... 
num vocabs = 225
done = 10
5
좋다
scan vocabs ... 
num vocabs = 262
done = 10 Early stopped.
5
연어
scan vocabs ... 
num vocabs = 181
done = 10
5
맛있다
scan vocabs ... 
num vocabs = 226
done = 10
5
맛있다
scan vocabs ... 
num vocabs = 372
done = 10
5
돈까스
scan vocabs ... 
num vocabs = 327
done = 10
5
하다
scan vocabs ... 
num vocabs = 330
done = 10
5
돈까스
scan vocabs ... 
num vocabs = 354
done = 10
5
음식
scan vocabs ... 
num vocabs = 394
done = 10
5
하다
scan vocabs ... 
num vocabs = 677
done = 10 Early stopped.
5
하다
scan vocabs ... 
num vocabs = 773
done = 10 Early stopped.
5
하다
scan vocabs ... 
num vocabs = 398
done = 10
5
버거
scan vocabs ... 
num vocabs = 270
done = 10
5
하다
scan vocabs ... 
num vocabs = 326
done = 10
5
먹다
scan vocabs ... 
num vocabs = 606
done = 10
5
하다
scan vocabs ... 
num vocabs = 321
done = 10
5
하다
scan vocabs ... 
num vocabs = 859
done = 10 Early stop

done = 10
5
커리
scan vocabs ... 
num vocabs = 596
done = 10 Early stopped.
5
떡볶
scan vocabs ... 
num vocabs = 906
done = 10 Early stopped.
5
하다
scan vocabs ... 
num vocabs = 386
done = 10
5
순대
scan vocabs ... 
num vocabs = 221
done = 10
5
하다
scan vocabs ... 
num vocabs = 375
done = 10
5
하다
scan vocabs ... 
num vocabs = 379
done = 10
5
하다
scan vocabs ... 
num vocabs = 634
done = 10
5
하다
scan vocabs ... 
num vocabs = 491
done = 10
5
하다
scan vocabs ... 
num vocabs = 429
done = 10
5
하다
scan vocabs ... 
num vocabs = 795
done = 10 Early stopped.
5
먹다
scan vocabs ... 
num vocabs = 577
done = 10
5
먹다
scan vocabs ... 
num vocabs = 137
done = 10
5
파스타
scan vocabs ... 
num vocabs = 827
done = 10 Early stopped.
5
하다
scan vocabs ... 
num vocabs = 206
done = 10
5
먹다
scan vocabs ... 
num vocabs = 172
done = 10
5
맵다
scan vocabs ... 
num vocabs = 981
done = 10 Early stopped.
5
버거
scan vocabs ... 
num vocabs = 716
done = 10 Early stopped.
5
하다
scan vocabs ... 
num vocabs = 685
done = 10 Early stopped.
5


num vocabs = 552
done = 10
5
늘다
scan vocabs ... 
num vocabs = 437
done = 10
5
하다
scan vocabs ... 
num vocabs = 228
done = 10
5
맛있다
scan vocabs ... 
num vocabs = 402
done = 10
5
하다
scan vocabs ... 
num vocabs = 331
done = 10
5
하다
scan vocabs ... 
num vocabs = 271
done = 10
5
하다
scan vocabs ... 
num vocabs = 372
done = 10
5
먹다
scan vocabs ... 
num vocabs = 266
done = 10
5
초밥
scan vocabs ... 
num vocabs = 473
done = 10
5
하다
scan vocabs ... 
num vocabs = 296
done = 10
5
파스타
scan vocabs ... 
num vocabs = 395
done = 10
5
맥주
scan vocabs ... 
num vocabs = 1021
done = 10 Early stopped.
5
전복
scan vocabs ... 
num vocabs = 329
done = 10
5
하다
scan vocabs ... 
num vocabs = 470
done = 10
5
부대
scan vocabs ... 
num vocabs = 289
done = 10
5
육회
scan vocabs ... 
num vocabs = 420
done = 10
5
버거
scan vocabs ... 
num vocabs = 314
done = 10
5
하다
scan vocabs ... 
num vocabs = 366
done = 10
5
하다
scan vocabs ... 
num vocabs = 623
done = 10
5
하다
scan vocabs ... 
num vocabs = 211
done = 10
5
하다
scan vocabs ... 
nu

done = 10
5
하다
scan vocabs ... 
num vocabs = 187
done = 10
5
맛있다
scan vocabs ... 
num vocabs = 850
done = 10 Early stopped.
5
하다
scan vocabs ... 
num vocabs = 321
done = 10
5
하다
scan vocabs ... 
num vocabs = 282
done = 10
5
하다
scan vocabs ... 
num vocabs = 292
done = 10
5
하다
scan vocabs ... 
num vocabs = 202
done = 10
5
하다
scan vocabs ... 
num vocabs = 319
done = 10
5
하다
scan vocabs ... 
num vocabs = 292
done = 10
5
피자
scan vocabs ... 
num vocabs = 320
done = 10
5
피자
scan vocabs ... 
num vocabs = 203
done = 10
5
만두
scan vocabs ... 
num vocabs = 762
done = 10
5
하다
scan vocabs ... 
num vocabs = 300
done = 10
5
하다
scan vocabs ... 
num vocabs = 243
done = 10
5
하다
scan vocabs ... 
num vocabs = 1204
done = 10 Early stopped.
5
만두
scan vocabs ... 
num vocabs = 235
done = 10
5
태국
scan vocabs ... 
num vocabs = 415
done = 10
5
하다
scan vocabs ... 
num vocabs = 92
done = 10
5
하다
scan vocabs ... 
num vocabs = 425
done = 10
5
하다
scan vocabs ... 
num vocabs = 330
done = 10
5
하다
scan vocabs ... 
num vo

done = 10
5
피자
scan vocabs ... 
num vocabs = 320
done = 10
5
늘다
scan vocabs ... 
num vocabs = 363
done = 10
5
하다
scan vocabs ... 
num vocabs = 321
done = 10
5
하다
scan vocabs ... 
num vocabs = 206
done = 10
5
맥주
scan vocabs ... 
num vocabs = 730
done = 10 Early stopped.
5
먹다
scan vocabs ... 
num vocabs = 1132
done = 10 Early stopped.
5
하다
scan vocabs ... 
num vocabs = 358
done = 10
5
먹다
scan vocabs ... 
num vocabs = 338
done = 10
5
먹다
scan vocabs ... 
num vocabs = 416
done = 10
5
하다
scan vocabs ... 
num vocabs = 351
done = 10
5
마르
scan vocabs ... 
num vocabs = 304
done = 10
5
하다
scan vocabs ... 
num vocabs = 547
done = 10 Early stopped.
5
하다
scan vocabs ... 
num vocabs = 399
done = 10
5
돈까스
scan vocabs ... 
num vocabs = 185
done = 10
5
와플
scan vocabs ... 
num vocabs = 237
done = 10
5
닭강
scan vocabs ... 
num vocabs = 1161
done = 10 Early stopped.
5
버거
scan vocabs ... 
num vocabs = 172
done = 10
5
물회
scan vocabs ... 
num vocabs = 1027
done = 10 Early stopped.
5
먹다
scan vocabs ... 
num voc

done = 10
5
국밥
scan vocabs ... 
num vocabs = 282
done = 10
5
하다
scan vocabs ... 
num vocabs = 206
done = 10
5
양념
scan vocabs ... 
num vocabs = 1105
done = 10 Early stopped.
5
하다
scan vocabs ... 
num vocabs = 215
done = 10
5
음식
scan vocabs ... 
num vocabs = 764
done = 10 Early stopped.
5
하다
scan vocabs ... 
num vocabs = 429
done = 10
5
밀면
scan vocabs ... 
num vocabs = 283
done = 10
5
하다
scan vocabs ... 
num vocabs = 362
done = 10
5
하다
scan vocabs ... 
num vocabs = 1512
done = 10 Early stopped.
5
하다
scan vocabs ... 
num vocabs = 289
done = 10
5
먹다
scan vocabs ... 
num vocabs = 407
done = 10
5
족발
scan vocabs ... 
num vocabs = 417
done = 10
5
하다
scan vocabs ... 
num vocabs = 868
done = 10 Early stopped.
5
라멘
scan vocabs ... 
num vocabs = 580
done = 10
5
하다
scan vocabs ... 
num vocabs = 316
done = 10
5
하다
scan vocabs ... 
num vocabs = 496
done = 10
5
하다
scan vocabs ... 
num vocabs = 253
done = 10
5
하다
scan vocabs ... 
num vocabs = 293
done = 10
5
먹다
scan vocabs ... 
num vocabs = 330
done = 

In [26]:
keywords, rank, graph = wordrank.extract(review_list[0]['review'], 0.85, 10)
print(next(iter(keywords.keys())))

scan vocabs ... 
num vocabs = 276
done = 10
좋다


In [43]:
from wordcloud import WordCloud


font_path = '/usr/share/fonts/truetype/nanum/NanumGothic_Coding.ttf'
wc = WordCloud(font_path=font_path, background_color='white', width=200, height=200)

In [27]:
import matplotlib.pyplot as plt

def save_img(wordcloud, filename):
    arr = wordcloud.to_array()
    fig = plt.figure(figsize=(5, 5))
    plt.imshow(arr, interpolation='bilinear')
    plt.axis("off")
    fig.savefig(filename)

In [44]:
keywords, rank, graph = wordrank.extract(review_list[0]['review'], 0.85, 10)

scan vocabs ... 
num vocabs = 276
done = 10


In [9]:
from tqdm import tqdm_notebook

for l in tqdm_notebook(review_list):
    filename = 'img/{}.png'.format(l['_id'])
    while True:
        try:
            keywords, rank, graph = wordrank.extract(l['review'], 0.85, 10)
        except:
            continue
        break
    wordcloud = wc.generate_from_frequencies(keywords)
    save_img(wordcloud, filename)

In [21]:
r = stat.find({"latitude": {"$exists": False}})

In [20]:
import requests
import json
from tqdm import tqdm_notebook

url = "https://dapi.kakao.com/v2/local/search/keyword.json?query={keyword}"
headers = {'Authorization': "KakaoAK 60d649aadfff4cef8a1d96e39d05cd2c"}

for restaurant in tqdm_notebook(r):
    name = restaurant['name']
    resp = requests.get(url.format(keyword=name), headers=headers)
    data = json.loads(resp.text)

    if len(data['documents']) < 1:
        print(name)
    else:
        loc = data['documents'][0]
        latitude = loc['y']
        longitude = loc['x']
        stat.update_one({'name': name}, {'$set': {'latitude': latitude, 'longitude': longitude}})
        

늘봄 흙돼지
젤렌 이태원 본점
오브라이언즈 아이리쉬 펍 & 레스토랑
버터핑커 팬케익 분당점
새마을 식당 논현 본점
이드 - 할랄 코리안 푸드
파라디소 페르두또 이탈리안 카페 & 키친
갈메기 브루어리
더 신더 바
샤키즈 바 앤 그릴 3
브릭 오븐 뉴욕 피자
울프강스테이크하우스코리아
제임스 치즈 등갈비 홍대 본점
데블스도어

